# Grabbing the database

StackOverflow.com is an amazing knowledgebase.   
That being said,  This notebook aims to create a local copy of the Python   
Questions and answers to free synapses for search engine caching engines.  
The dataset is graciously hosted by the folks at Kaggle.com and 
All Stack Overflow user contributions are licensed under CC-BY-SA 3.0 with attribution required.  
https://creativecommons.org/licenses/by-sa/3.0/  
https://stackoverflow.com  

In [ ]:
import requests, shutil

In [ ]:
BigZipResponse = requests.get('https://www.kaggle.com/stackoverflow/pythonquestions/downloads/pythonquestions.zip')

In [ ]:
with open('pythonquestions.zip', 'wb') as f:
    shutil.copyfileobj(r.raw, f)

# Importing the files

In [2]:
import pymongo

Start the mongo daemon
```bash
mongod
```

In [3]:
client = pymongo.MongoClient()
databases = client.database_names()
database = client['PythonSODB']
database.create_collection('PythonQuestions')
database.create_collection('PythonAnswers')
database.create_collection('Tags')

In [72]:
import re, copy, os, codecs

In [ ]:
collection = database['PythonQuestions']

In [194]:
def SODBToMongo(input_csv, collection, answers=True):
    answers_fields = 'Id,OwnerUserId,CreationDate,ParentId,Score,Body'.split(',')
    questions_fields = 'Id,OwnerUserId,CreationDate,Score,Title,Body'.split(',')
    if answers == True:
        fields = answers_fields
    else:
        fields = questions_fields
    f2 = open(input_csv, 'r')
    f = codecs.open(input_csv, "r",encoding='utf-8', errors='ignore')
    file_chunk = 65536
    so_entry = re.compile('''(\d+),(\d+),(\d+\S+?),(\d+),([\S\s]+?),(.*?)\n\d+,\d+,\d{4}''', re.DOTALL)
    header_offset = f.readline()
    header_offset = len(header_offset)
    current_offset = header_offset
    max_offset = f2.seek(0, os.SEEK_END)
    f2.close()
    f.seek(header_offset, 0)
    counter = 0
    while current_offset < max_offset:
        try:
            work_file = codecs.open(input_csv, "r",encoding='utf-8', errors='ignore')
            work_file.seek(current_offset, 0)
            file_chunk = work_file.read(65536)
            entry_values = so_entry.search(file_chunk)
            entry_list = entry_values.groups()
            current_offset += len(','.join(entry_list))
            entry = {}
            entry[fields[0]] = entry_list[0]
            entry[fields[1]] = entry_list[1]
            entry[fields[2]] = entry_list[2]
            entry[fields[3]] = entry_list[3]
            entry[fields[4]] = entry_list[4]
            entry[fields[5]] = entry_list[5]
            collection.insert_one(entry)
            percent_complete = (current_offset / max_offset) * 100
            counter += 1
            if counter % 10000 == 0:
                print('{}% of {} imported to {}'.format(percent_complete, input_csv, collection.name))
        except:
            return "End of File Reached"

In [ ]:
SODBToMongo('Questions.csv', collection, answers=False)

1.1861742238326771% of Questions.csv imported to PythonQuestions
2.3758887287340316% of Questions.csv imported to PythonQuestions
3.560339207979959% of Questions.csv imported to PythonQuestions


In [192]:
collection = database['PythonAnswers']
SODBToMongo('Answers.csv', collection, answers=True)

['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body']
0.8727682072551932% of Answers.utf8.csv imported to PythonAnswers
1.7323076061806386% of Answers.utf8.csv imported to PythonAnswers
2.6289880016785405% of Answers.utf8.csv imported to PythonAnswers
3.521581849076276% of Answers.utf8.csv imported to PythonAnswers
4.385918500119308% of Answers.utf8.csv imported to PythonAnswers
5.24166031851211% of Answers.utf8.csv imported to PythonAnswers
6.131862495573751% of Answers.utf8.csv imported to PythonAnswers
6.989917518869405% of Answers.utf8.csv imported to PythonAnswers
7.881396254091457% of Answers.utf8.csv imported to PythonAnswers
8.716353396820956% of Answers.utf8.csv imported to PythonAnswers
9.562498148343908% of Answers.utf8.csv imported to PythonAnswers
10.441118436962322% of Answers.utf8.csv imported to PythonAnswers
11.294696049354275% of Answers.utf8.csv imported to PythonAnswers
12.156469333537771% of Answers.utf8.csv imported to PythonAnswers
13.022073146414058

'End of File Reached'

In [ ]:
def tagsImport(tags_file, collection):
    reg = re.compile('(\d+),(\S+)\n')
    f = open(tags_file, 'r')
    f.readline()
    counter = 0
    for i,l in enumerate(iter(f)):
        pass
    end = i+1
    f.seek(0, 0)
    for line in f:
        data = f.readline()
        data = reg.search(data).groups()
        collection.insert_one({'Id':data[0], 'Tags':data[1]})
        counter +=1
        if counter%10000 == 0:
            print('{}% of tags written to {}'.format(counter/end * 100, collection.name))
    return('All tags written to {}'.format(collection.name))

In [ ]:
collection = database['Tags']
tagsImport('Tags.csv', collection)

TODO
* Need to write a loop to grab the last entry in the file